In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import altair as alt
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import random
import re
from collections import Counter, defaultdict
from nltk.tokenize import word_tokenize
from sklearn.metrics import f1_score
import nltk
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_positve=pd.read_excel('refined_dict_eval_df_Positive.xlsx')
df_positve['table id']=1
df_VnEmoLex_validated=pd.read_excel('eval_df_NlpHUST_emotion 4 43 pm Jul 15 2022.xlsx')
df_VnEmoLex_validated['table id']=2
df_extra_emotion_data_1=pd.read_excel('extra_pos_neg_train_review_NlpHUST_NLP_Epoch10_eval_df 6 47 pm Jul 15 2022.xlsx')
df_extra_emotion_data_1['table id']=3
df_extra_emotion_data_2=pd.read_excel('extra_pos_neg_train_review_NlpHUST_NLP_Epoch2_eval_df 11 20pm Jul 15 2022.xlsx')
df_extra_emotion_data_2['table id']=4
df_extra_emotion_data_3=pd.read_excel('train_review_NlpHUST_NLP_Epoch10_eval_df 8 52 am Jul 16 2022.xlsx')
df_extra_emotion_data_3['table id']=5

In [3]:
df_combined=pd.concat([df_positve,df_VnEmoLex_validated,df_extra_emotion_data_1,df_extra_emotion_data_2,df_extra_emotion_data_3])
df_combined_long=pd.melt(df_combined, id_vars=['emotion', 'epoch','batch_size', 'Ir', 'eps',
       'pre_trained_model', 'hidden_dropout_prob',
       'attention_probs_dropout_prob', 'Note', 'table id'], value_vars=['train_loss', 'val_loss', 'F1_score_weighted',
       'F1_score_macro', 'F1_score_micro'],var_name=['line type'])
df_combined_long.head(2)


,emotion,epoch,batch_size,Ir,eps,pre_trained_model,hidden_dropout_prob,attention_probs_dropout_prob,Note,table id,line type,value
0,positive,1,2,0.00001,1.000000e-08,NlpHUST/vibert4news-base-cased,0.1,0.1,added VnEmoLex _Validate to train daa,1,train_loss,0.921794
1,positive,2,2,0.00001,1.000000e-08,NlpHUST/vibert4news-base-cased,0.1,0.1,added VnEmoLex _Validate to train daa,1,train_loss,0.868497


In [4]:
width=150
height=120
scheme_name='greens'
domain=(0,2.62)
def draw_graph(df,domain,table_id,width,height,scheme_name,
               first_line_title,second_line_title,third_line_title,fourth_line_title):
    df=df[df['table id']==table_id]

    base = alt.Chart(df).mark_line().encode(
        x='epoch'
    ).properties(
        width=width,
        height=height,
        title={
          "text": [first_line_title,second_line_title,third_line_title,fourth_line_title], 
          #"subtitle": [subtitle1, subtitle2],
          "color": "black",
          "subtitleColor": "black"
        }
    )
    graph=base.mark_line().encode(
        y=alt.Y("value:Q",title="",scale=alt.Scale(domain=domain)),
        color=alt.Color('line type:N',
                        #, scale=alt.Scale(scheme=scheme_name),
                        legend=alt.Legend(
                        orient='bottom',
                        titleFontSize=11,
                        titleColor='black',
                        labelFontSize=10.5,
                        labelColor='black',
                        direction='horizontal'
                        )
                       ),
        tooltip=['emotion','epoch','line type','value']
        ).interactive()
    return graph

In [5]:
def combined_graph(df):
    graph1=draw_graph(df,domain,1,width,height,scheme_name,'Positive','in VnEmoLex',"","")
    graph2=draw_graph(df,domain,2,width,height,scheme_name,'Positive', 'in VnEmoLex_Validate',"","")
    graph3=draw_graph(df,domain,3,width,height,scheme_name,'Positive, Negative, Neutral','in VnEmoLex_Validate','Extra Eng-Viet Emotions','Extra Review Emotions')
    graph4=draw_graph(df,domain,4,width,height,scheme_name,'Positive, Negative, Neutral','No Validated VnEmolex','Extra Eng-Viet emotions', 'Extra Review Emotions')
    graph5=draw_graph(df,domain,5,width,height,scheme_name,'Positive, Negative, Neutral','No Validated VnEmolex','No Eng-Viet emotions', 'Extra Review Emotions')

    chart=(graph1 | graph2 | graph3 | graph4 | graph5).configure_view(
        stroke=None
    ).configure_concat(
        spacing=6
    ).configure_title(fontSize=11)
    return chart
combined_graph(df_combined_long)

alt.HConcatChart(...)

In [6]:
df_combined_long1=df_combined_long[df_combined_long['epoch']<3]
domain=(0,1.2)
combined_graph(df_combined_long1)

alt.HConcatChart(...)

The BERT model needs to be continously fined tuned